## 2.4 Model Validation and application
### 2.4.3 Heterotrophic metabolism is redox-limited

This notebook recreates the analyses and figures from section 2.4.3. Flux sampling of the model[1] using Methane, Propane, Isopropanol and Acetone. This analysis is divided in two notebooks, one dedicated to performing the simulations (This) and other dedicted to plotting and data analysis.

The notebook is divided in three sections:
1. In the first section we pre-process the model to reduce computational burden of the simulations
2. The second section runs independent sampling using OptGPSampler [1]
3. Finally, for each simulation we calculate the PageRank score for each reactiong using Mass Flow Graphs calculated following the methods presentend in [2]

[1] Megchelenbrink W, Huynen M, Marchiori E (2014) optGpSampler: An Improved Tool for Uniformly Sampling the Solution-Space of Genome-Scale Metabolic Networks. PLoS ONE 9(2): e86587. https://doi.org/10.1371/journal.pone.0086587

[2] Beguerisse-Díaz, M., Bosque, G., Oyarzún, D. et al. Flux-dependent graphs for metabolic networks. npj Syst Biol Appl 4, 32 (2018). https://doi.org/10.1038/s41540-018-0067-y

In [1]:
# Set up the modeling environment
from tqdm import tqdm
from multiprocessing import Pool

from cobra.sampling import OptGPSampler
from helpers import find_blocked_mets, build_normilized_flow_graph, build_mass_flow_graph

import scipy.stats as stats
import pandas as pd
import numpy as np
import networkx as nx
import json

import cobra

# Change solvers as available
# Recomended solvers: 'cplex', 'gurobi'
cobra_config = cobra.Configuration()
cobra_config.solver = 'cplex'
cobra_config.tolerance = 1e-6

## Section 1 
---

Pre-processing of the model. This cell prints a list of metabolites and reactions removed from the model to reduce computational burden.

In [2]:
# Load the model
model = cobra.io.read_sbml_model('../model_files/iMFP2023.xml')
model.solver.configuration.lp_method = 'auto'

# Remove unused reactions
to_remove = ['N2tex', 'N2trpp', 'NIT1c']
for id in to_remove:
    rxn = model.reactions.get_by_id(id)
    model.remove_reactions([rxn])

# Identify and remove structuraly blocked metabolites and their associated reactions
orphans, deadends = find_blocked_mets(model)

blocked_mets = orphans.copy()
blocked_mets.extend(deadends)
blocked_mets.extend(['ditp_c', 'cu_e', 'ppoh_e', 'cbl1_e'])
blocked_mets = list(pd.unique(blocked_mets))

while blocked_mets:
    for id in blocked_mets:
        
        met = model.metabolites.get_by_id(id)
        print(f"{id}: ", end="")
        
        for rxn in met.reactions:
            
            model.remove_reactions([rxn])
            print(f"{rxn.id}, ", end="")
            
        model.remove_metabolites([met])
        print("")
        
    orphans, deadends = find_blocked_mets(model)
    
    blocked_mets = orphans.copy()
    blocked_mets.extend(deadends)
    blocked_mets = list(pd.unique(blocked_mets))

# remove constrained reactions
for rxn in model.reactions:
    lb = rxn.lower_bound
    ub = rxn.upper_bound
    if lb == ub:
        model.remove_reactions([rxn])
        print(rxn.id)
        
for rxn in model.reactions:
    lb = rxn.lower_bound
    ub = rxn.upper_bound
    if lb == ub:
        model.remove_reactions([rxn])
        print(rxn.id)
        
for rxn in model.reactions:
    lb = rxn.lower_bound
    ub = rxn.upper_bound
    if lb == ub:
        model.remove_reactions([rxn])
        print(rxn.id)

S2hglut_c: SHGO, 
mmet_c: HCYSMT2, 
fe3_p: FE3abcpp, 
gsn_c: PUNP3, 
hg2_p: HG2tppi, 
4ahmmp_c: HMPK1, 
malthx_c: AMALT3, 
ncam_c: NNAM, 
n2_c: 
pmtcoa_c: FACOAE160, 
pyam5p_c: PYAM5PO, 
stcoa_c: FACOAE180, APG3PAT180, 
o2s_c: SPODM, 
xtsn_c: PUNP7, 
n2_p: 
n2_e: EX_n2_e, 
ch4s_c: MTOX, 
fe3_c: 
hg2_c: 
malt_c: AMALT2, AMALT1, MLTG1, 
xan_c: 
hemeA_c: HEMEAS_2, 
ditp_c: NTPP10, RNTRA5, RNTR5, 
cu_e: CUt3, 
ppoh_e: PPOHtex, 
cbl1_e: CBL1tex, 
cu_c: 
dimp_c: 
maltpt_c: 
maltttr_c: 
malttr_c: 
nac_c: NAPRT, 
cbl1_p: CBL1abcpp, 
cbl1_c: CBL1MT, 
FHL
PO2OXG
Kt2pp
NAD_H2
ZN2t3pp
NAt3pp
FE2t3pp
PPANAp
CU2t3pp
MG2t3_2pp
HYD4pp
Kt3pp


##  Section 2 
---

This section setups and runs optgp_sampler. Parameters to define are:
- Carbon uptake rate (C-mmol gDW-1 h-1)
- The flux ration FALDHpp / EX_ch4_e used in methane simulations
- The number of samples in each condition
- The number of parallel workers to use in each indepent sampling

optgp_sampler uses a Hit-and-Run sampling algorithm; therefore, results won't be exactly the same between independent runs.
Increasing the number of samples increases the time required for the simulations. For reference, 10,000 samples for the four conditions took ~10min in a Ryzen 7 gen 2 using 4 workers.

In [6]:
# Parameters used in Methane simulations
carbon_rate = 3.5
faldhpp = 0.2

# Parameters used in all simulations
samples = 1000
parallel_workers=4

cmol_by_mol = 3
valid_samples = {}

# Open a model instance to make changes reversible
with model as instance:
    
    # Set the Carbon uptake rate
    medium = instance.medium
    medium["EX_ch4_e"] = carbon_rate
    instance.medium = medium
    
    # Constraint ratio FALDHpp / EX_ch4_e
    instance.reactions.FALDHpp.lower_bound = faldhpp * carbon_rate
    
    # Flux sampling
    optgp_sampler = OptGPSampler(instance, processes=parallel_workers)
    samples_ch4 = optgp_sampler.sample(samples)
    
    # Recover only valid samples (mass balanced samples)
    valid_samples["Methane"] = samples_ch4[optgp_sampler.validate(samples_ch4) == "v"]

with model as instance:
    
    # Set the Carbon uptake rate
    medium = instance.medium
    substrate_rate = carbon_rate / cmol_by_mol
    medium['EX_c3h8_e'] = substrate_rate
    instance.medium = medium

    # Flux sampling
    optgp_sampler = OptGPSampler(instance, processes=parallel_workers)
    samples_c3h8 = optgp_sampler.sample(samples)
    
    # Recover only valid samples (mass balanced samples)
    valid_samples["Propane"] = samples_c3h8[optgp_sampler.validate(samples_c3h8) == "v"]

with model as instance:
    
    # Set the Carbon uptake rate
    medium = instance.medium
    substrate_rate = carbon_rate / cmol_by_mol
    medium['EX_2ppoh_e'] = substrate_rate
    instance.medium = medium

    # Flux sampling
    optgp_sampler = OptGPSampler(instance, processes=parallel_workers)
    samples_ppoh = optgp_sampler.sample(samples)
    
    # Recover only valid samples (mass balanced samples)
    valid_samples["Isopropanol"] = samples_ppoh[optgp_sampler.validate(samples_ppoh) == "v"]

with model as instance:
    
    # Set the Carbon uptake rate
    medium = instance.medium
    substrate_rate = carbon_rate / cmol_by_mol
    medium['EX_acetone_e'] = substrate_rate
    instance.medium = medium

    # Flux sampling
    optgp_sampler = OptGPSampler(instance, processes=parallel_workers)
    samples_acetone = optgp_sampler.sample(samples)
    
    # Recover only valid samples (mass balanced samples)
    valid_samples["Acetone"] = samples_acetone[optgp_sampler.validate(samples_acetone) == "v"]

In [7]:
# This Cell calculates median values for each reaction, the log2 foldchange for each condition using methane
# as a reference, and summary statistics

summary_tests = {}
directionality = []
writer = pd.ExcelWriter('data_files/summary_sampling.xlsx')

for substrate in ["Propane", "Isopropanol", "Acetone"]:
    
    summary_tests[substrate] = pd.DataFrame()
    
    for id in valid_samples["Methane"].columns:
        if "EX_ch4_e" not in id:
            
            # Calculate median value for each reaction
            median_ch4 = np.median(valid_samples["Methane"].loc[:, id])
            median_substrate = np.median(valid_samples[substrate].loc[:, id])
            
            # Calculate the flux foldchange
            # When reactions have the same directionality
            if abs(median_substrate * median_ch4) > 0:
                foldchange = median_substrate / median_ch4
                
            # When reactions change directionality
            elif abs(median_substrate * median_ch4) < 0:
                directionality.append(id)
                foldchange = median_substrate / median_ch4
                
            # When reaction flux is zero using Methane
            elif abs(median_substrate) > 0:
                foldchange = 2
                
            # When reaction flux is zero using 'Substrate'
            else:
                foldchange = 0.5
            
            # For reactions with the same directionality
            if foldchange < 0:
                directionality.append(id)
                
            else:
                
                # Calculate the log2 foldchange
                log2F = np.log2(foldchange)
                
                # Calculate summary statistics and tests values
                wasserstein = stats.wasserstein_distance(
                    valid_samples[substrate].loc[:, id], valid_samples["Methane"].loc[:, id]
                )
                energy = stats.energy_distance(
                    valid_samples[substrate].loc[:, id], valid_samples["Methane"].loc[:, id]
                )
                moods = stats.ks_2samp(
                    valid_samples[substrate].loc[:, id], valid_samples["Methane"].loc[:, id]
                )
                
                try:
                    kruskal = stats.kruskal(
                        valid_samples[substrate].loc[:, id], valid_samples["Methane"].loc[:, id]
                    )
                    summary_tests[substrate].loc[id, "kruskal"] = kruskal.statistic
                except ValueError:
                    summary_tests[substrate].loc[id, "kruskal"] = 0
                    continue
                    
                ttest = stats.ttest_ind(
                    valid_samples[substrate].loc[:, id], valid_samples["Methane"].loc[:, id], equal_var=False
                )
                kstest = stats.ks_2samp(
                    valid_samples[substrate].loc[:, id], valid_samples["Methane"].loc[:, id]
                )

                # Save the results
                summary_tests[substrate].loc[id, "median_ch4"] = median_ch4
                summary_tests[substrate].loc[id, "median"] = median_substrate
                summary_tests[substrate].loc[id, "log2F"] = log2F
                summary_tests[substrate].loc[id, "wasserstein"] = wasserstein
                summary_tests[substrate].loc[id, "energy"] = energy
                summary_tests[substrate].loc[id, "moods"] = moods[0]
                summary_tests[substrate].loc[id, "kruskal"] = kruskal.statistic
                summary_tests[substrate].loc[id, "ttest"] = ttest.statistic
                summary_tests[substrate].loc[id, "kstest"] = kstest.statistic
                
    summary_tests[substrate].to_excel(writer, sheet_name=substrate)
    
writer.save()

## Section 3

For each simulation calculate a Mass Flow Graph (MFG). MFGs are wighted directed networks in which nodes represent reactions, edges represent supplier-consumer relationships between reactions, and weights given by the mass flow between connected reactions. PageRank values are an indicator of the centrality of the reactions that intagrates the connectivity and the mass flow.

**Important note: this is the most computationally intesive section of the notebook. For reference, calculating PageRank values for 10,000 simulations for the 4 conditions took ~4hrs in a Ryzen 7 gen 2.**

In [8]:
workers = 4
chunksize = 1

# The method used to calculate MFG discriminates between forward and reversible reactions,
# Here we create a list of forward and reversible reactions used for indexing of the results

r_f = []
r_r = []
for rxn in model.reactions:
    r_f.append(rxn.id)
    r_r.append(rxn.id + "_r")

rxn_id = r_f.copy()
rxn_id.extend(r_r)

# Create a model instance to make all changes reversible
with model as instance:
    
    # Pre-process the model
    medium = instance.medium
    medium["EX_ch4_e"] = carbon_rate
    medium['EX_c3h8_e'] = substrate_rate
    medium['EX_2ppoh_e'] = substrate_rate
    medium['EX_acetone_e'] = substrate_rate
    instance.medium = medium
    m = len(instance.reactions)
    
    # Calculate the normalized flow graph of the model, and matrices S2m_p, and S2m_c used
    # as input to the function calculating MFGs and PageRank values
    NFG, S2m_p, S2m_c = build_normilized_flow_graph(instance, tol=1e-3)

    # Set up the list of inputs as required by the parallel processing package
    # Each element of the list contains: (Solution[i], S2m_p, S2m_c)
    items = [(valid_samples["Methane"].loc[s, :].to_numpy(), S2m_p, S2m_c) for s in valid_samples["Methane"].index]
    
    pagerank_ch4 = pd.DataFrame(columns=rxn_id)
    with Pool(workers) as pool:
        with tqdm(total=samples) as pbar:
            
            # Parallel processing of each solution
            for pagerank in pool.imap_unordered(build_mass_flow_graph, items, chunksize=chunksize):
                pagerank.columns = rxn_id
                pagerank_ch4 = pagerank_ch4.append(pagerank)
                pbar.update()
    pagerank_ch4.to_csv("data_files/pagerank_Methane.csv")
    
    # Set up the list of inputs as required by the parallel processing package
    # Each element of the list contains: (Solution[i], S2m_p, S2m_c)
    items = [(valid_samples["Propane"].loc[s, :].to_numpy(), S2m_p, S2m_c) for s in valid_samples["Propane"].index]

    pagerank_propane = pd.DataFrame(columns=rxn_id)
    with Pool(workers) as pool:
        with tqdm(total=samples) as pbar:
            
            # Parallel processing of each solution
            for pagerank in pool.imap_unordered(build_mass_flow_graph, items, chunksize=chunksize):
                pagerank.columns = rxn_id
                pagerank_propane = pagerank_propane.append(pagerank)
                pbar.update()
    pagerank_propane.to_csv("data_files/pagerank_Propane.csv")

    # Set up the list of inputs as required by the parallel processing package
    # Each element of the list contains: (Solution[i], S2m_p, S2m_c)
    items = [(valid_samples["Isopropanol"].loc[s, :].to_numpy(), S2m_p, S2m_c) for s in valid_samples["Isopropanol"].index]
    
    pagerank_propanol = pd.DataFrame(columns=rxn_id)
    with Pool(workers) as pool:
        with tqdm(total=samples) as pbar:
            
            # Parallel processing of each solution
            for pagerank in pool.imap_unordered(build_mass_flow_graph, items, chunksize=chunksize):
                pagerank.columns = rxn_id
                pagerank_propanol = pagerank_propanol.append(pagerank)
                pbar.update()
    pagerank_propanol.to_csv("data_files/pagerank_Isopropanol.csv")

    # Set up the list of inputs as required by the parallel processing package
    # Each element of the list contains: (Solution[i], S2m_p, S2m_c)
    items = [(valid_samples["Acetone"].loc[s, :].to_numpy(), S2m_p, S2m_c) for s in valid_samples["Acetone"].index]

    pagerank_acetone = pd.DataFrame(columns=rxn_id)
    with Pool(workers) as pool:
        with tqdm(total=samples) as pbar:
            
            # Parallel processing of each solution
            for pagerank in pool.imap_unordered(build_mass_flow_graph, items, chunksize=chunksize):
                pagerank.columns = rxn_id
                pagerank_acetone = pagerank_acetone.append(pagerank)
                pbar.update()
    pagerank_acetone.to_csv("data_files/pagerank_Acetone.csv")

Sum of probabilities (0.9984202211690364) below tolerance level 0.001
Remove blocked metabolites and reactions from the model


100%|███████████████████████████████████████| 1000/1000 [05:54<00:00,  2.82it/s]
